Changes in work formalisation and Moments of work revision sub-indicators are calculated monthly based on email data. The goal is to detect/extract organizational and revision patterns in samples in email communication.

Please use Tool_for_extracting_filtering_and_cleaning_GMAIL_data.ipynb for extracting data from mbox file and saving it to csv format.

In [ ]:
#install and imprort needed libraries
!pip install --upgrade gensim
!pip install --upgrade language_tool_python
!pip install googletrans==4.0.0-rc1
from re import sub
from gensim.utils import simple_preprocess
import numpy as np
import gensim.downloader as api
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.similarities import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix
from gensim.similarities import SoftCosineSimilarity
# Load the model: this is a big file, can take a while to download and open
glove = api.load("glove-wiki-gigaword-50")
similarity_index = WordEmbeddingSimilarityIndex(glove)
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
!pip install langdetect
from langdetect import detect
import language_tool_python
tool = language_tool_python.LanguageTool('en-US')
from googletrans import Translator
translator = Translator()


These sub-indicators require special approach to the preprocessing, because we need to clean emails text from noise that may impact the quality of semantic search that we use for organizational and revision patterns detection. This part was done manually so it's need to be automitised. The issue here is the fact that we are working with many languages (English, French, Italian etc).

Preprocessing steps:

1. remove signatures: 
2. remove "best regards synonyms". À très bientôt en chair et en os,
3. remove URls/emails and ">" symbols
4. proper names but not inside the sentences, only at the begging and at the end of email, including abbreviations at the end of message like T/DB
5. replay comments also should be removed till the end, keeping only first original message: Le ven. 18 mars 2022 à 08:26, XXXXX < XXXXX@XXXXX> a écrit/ On Mon, May 2, 2022 at 8:47 AM XXXXX@XXXXX wrote:/ ---------- Forwarded message ---------

6. remove greetings: Dear XXXX, dear XXXX, Bonjour XXXX, Dear COESO WP4  and WP2  Colleagues,Dear All
replies info inside letter: From: XXXXXX XXXXX@XXXXX Date: Tuesday, 8 March 2022 at 17:33 To: XXXX XXXXX@XXXX.es Subject: Pregunta para proyecto COESO Citizen Science


In [ ]:
# reading .csv cleaned dataset
import pandas as pd
data = pd.read_csv("put_csv_data_here.csv")

# previewing the first 5 lines of the loaded data
data.head()

#Translate text to English, we do it to have all communication in English

data['Translated'] = data['Body'].apply(lambda x: translator.translate(x, dest='en').text)
data

In [ ]:
# this step is needed to sort data month by month, but in real life data will come month by month and you will process it at the end of a given month.
month_sort = data.loc[data.Date.str.contains('Jun', na=False)]    #Sep, Oct, Nov, Dec, Jan, Feb, Mar, Apr, May, Jun, Jul, Aug
text = month_sort["Translated"].to_list()
print("original values: ")
print(len(text))

def remove_duplicates(sents):
  unique_sents = []
  for sent in sents:
    if sent not in unique_sents:
      unique_sents.append(sent)
  return unique_sents

subjects = remove_duplicates(text)
print("without duplicates: ")
print(len(subjects))


#additional cleaning to make sure that we removed all noisy text
import re
import string
from six.moves.html_parser import HTMLParser
h = HTMLParser()
target = string.printable + "öäüÖÄÜ"

def clean(value):
  sentences = []
  for v in subjects:
    if type(v) is not str:
        continue
    doc = re.sub ("\n", " ", v)
    doc = re.sub ("\r", " ",doc)
    doc = re.sub("http\S+", " ", doc) #removing urls
    doc = re.sub("[`~!@#$%^&*()_|+\-=?;:'<>\{\}\[\]\\\/]", ' ', doc)
    doc = h.unescape(doc) #converting other HTML entities to recongisable characters
    doc = re.sub("(\s|\t){2,}", " ", doc) #removing unnecessary spaces
    doc = re.sub("\S*@\S*\s?", " ", doc) #removing emails
    doc = re.sub("@[^\d]", " ", doc) #removing phone numbers
    sentences.append(doc.strip()) #removing leading and trailing spaces and adding clean string to the list
  return sentences


clean_sents = clean(subjects)
print("clean_sents")
print(len(clean_sents))


###tokenising text into sentences
full_text = '.'.join(str(x) for x in clean_sents)
splitted_sentences = nltk.sent_tokenize(full_text)

print("splitted_sentences")
print(len(splitted_sentences))

###creating new table
pd.set_option('display.max_colwidth', 0)
pd.set_option('display.max_columns', 0)
documents_df=pd.DataFrame(splitted_sentences,columns=['splitted_sentences'])
df_new = documents_df[documents_df['splitted_sentences'].notnull()]


text_messag = df_new['splitted_sentences'].to_list()
print("only eng")
print(len(text_messag))


###removing duplicates
new_subjects = remove_duplicates(text_messag)
print("second round check without duplicates:")
print(len(new_subjects))


SIMILARITY CHECK (Soft Cosine Measure):
we use semantic check to see how close semantically are samples out of emails and our target vocabularies of organizational and revision work

In [ ]:
###dictionary with contextual phrases###
d = dict()
d['organizational'] = ["Work organisation thus refers to how work is planned, organised and managed within companies and to choices on a range of aspects such as work processes, job design, responsibilities, task allocation, work scheduling, work pace, rules and procedures, and decision-making processes."]
d['revision'] = ["revision", "revise", "redo", "revising", "re-", "revised", "redone", "redraft"]

In [ ]:
samples = {}
for (name, v) in d.items():
    query_string = str(v)

    documents = new_subjects


    def preprocess(doc):
        # Tokenize, clean up input document string
        doc = sub(r'<img[^<>]+(>|$)', " image_token ", doc)
        doc = sub(r'\n', '', doc)
        doc = sub(r'http\S+', '', doc)
        doc = sub(r'<[^<>]+(>|$)', " ", doc)
        doc = sub(r'\[img_assist[^]]*?\]', " ", doc)
        doc = sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', " url_token ", doc)
        return [token for token in simple_preprocess(doc, deacc=False, min_len=2, max_len=15)]

    # Preprocess the documents, including the query string
    corpus = [preprocess(document) for document in documents]
    query = preprocess(query_string)
    raw_corpus = len(corpus)
    print(raw_corpus)

    # Build the term dictionary, TF-idf model
    dictionary = Dictionary(corpus+[query])
    tfidf = TfidfModel(dictionary=dictionary)

    # Create the term similarity matrix.
    similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)


    # Compute Soft Cosine Measure between the query and the documents.
    query_tf = tfidf[dictionary.doc2bow(query)]

    index = SoftCosineSimilarity(
                tfidf[[dictionary.doc2bow(document) for document in corpus]],
                similarity_matrix)

    doc_similarity_scores = index[query_tf]

    # Output the sorted similarity scores and documents
    sorted_indexes = np.argsort(doc_similarity_scores)[::-1]
    samples[name] = 0
    print(name)
    for idx in sorted_indexes:
        if doc_similarity_scores[idx] > 0.3:
            samples[name] = samples[name] + 1
print(samples)


In [ ]:
print(samples) #count of samples where the model detected implicit and explicit markers of organizaional and revision work
len_org = samples.get("organizational")
len_rev = samples.get("revision")
print(len_org)
print(len_rev)

In [ ]:
#calculate the percentage of samples with implicit and explicit markers of organizational and revision work out of the total count of samples and present them as fraction of 1
def is_what_percent_of(num_a, num_b):
    return (num_a / num_b) * 100

org_percent = is_what_percent_of (len_org, raw_corpus)
organization = org_percent / 100
print("Changes in work formalisation is {:0.2f}.".format(organization))

rev_percent = is_what_percent_of (len_rev, raw_corpus)
revision = rev_percent /100
print("Moments of work revision is {:0.2f}.".format(revision))

#No additional normalization is needed, as we are already in the scale from 0 to 1
